## TF-IDF ##

In [85]:
from pyspark.mllib.feature import HashingTF, IDF
from pyspark.mllib.common import callMLlibFunc, JavaModelWrapper
from pyspark.mllib.linalg.distributed import RowMatrix, DistributedMatrix

In [113]:
documents = sc.textFile("data_works.txt").map(lambda line: line.split(" "))

hashingTF = HashingTF()
tf = hashingTF.transform(documents)

tf.cache()
idf = IDF().fit(tf)
tfidf = idf.transform(tf)


#idTerms = idTerms.groupByKey().collect()
#print idTerms.collect()
#idfIgnore = IDF(minDocFreq=2).fit(tf)
#tfidfIgnore = idfIgnore.transform(tf)

<class 'pyspark.rdd.PipelinedRDD'>


In [87]:
class SVD(JavaModelWrapper):
    """Wrapper around the SVD scala case class"""
    @property
    def U(self):
        """ Returns a RowMatrix whose columns are the left singular vectors of the SVD if computeU was set to be True."""
        u = self.call("U")
        if u is not None:
            return RowMatrix(u)

    @property
    def s(self):
        """Returns a DenseVector with singular values in descending order."""
        return self.call("s")

    @property
    def V(self):
        """ Returns a DenseMatrix whose columns are the right singular vectors of the SVD."""
        return self.call("V")

def computeSVD(row_matrix, k, computeU=True, rCond=1e-9):
    """
    Computes the singular value decomposition of the RowMatrix.
    The given row matrix A of dimension (m X n) is decomposed into U * s * V'T where
    * s: DenseVector consisting of square root of the eigenvalues (singular values) in descending order.
    * U: (m X k) (left singular vectors) is a RowMatrix whose columns are the eigenvectors of (A X A')
    * v: (n X k) (right singular vectors) is a Matrix whose columns are the eigenvectors of (A' X A)
    :param k: number of singular values to keep. We might return less than k if there are numerically zero singular values.
    :param computeU: Whether of not to compute U. If set to be True, then U is computed by A * V * sigma^-1
    :param rCond: the reciprocal condition number. All singular values smaller than rCond * sigma(0) are treated as zero, where sigma(0) is the largest singular value.
    :returns: SVD object
    """
    
    java_model = row_matrix._java_matrix_wrapper.call("computeSVD", int(k), computeU, float(rCond))
    return SVD(java_model)

In [88]:
l = tfidf.collect()
#print l
rowM = RowMatrix (tfidf) 
matrixSVD = computeSVD(rowM,True)

#for i in l:
 #   print i


In [92]:
#print matrixSVD.U.rows.collect()

[DenseVector([-0.0]), DenseVector([-0.9924]), DenseVector([-0.0141]), DenseVector([-0.0329]), DenseVector([-0.0114]), DenseVector([-0.0835]), DenseVector([-0.0]), DenseVector([-0.0002]), DenseVector([-0.0541]), DenseVector([-0.0021]), DenseVector([-0.0]), DenseVector([-0.0267]), DenseVector([-0.0]), DenseVector([-0.0544]), DenseVector([-0.0145])]


In [130]:
v = matrixSVD.V
arr = v.toArray()




topTerms = []


#for i in range(0, v.numCols):
    #offs = i * v.numRows
    #termWeights = arr[offs : offs+v.numRows]
    #sortWeights =  termWeights.sort()
    
    #sortedWeights = termWeights.sort()
    #topTerms.append(sortedWeights)
    

NameError: name 'copy' is not defined